In [ ]:
from IPython.core.display import HTML
with open('style.css') as file:
    css = file.read()
HTML(css)

# Introduction

## Research Objectives

The main goal of this work is the development of a fully functional chess engine. One of the main parts, the implementation of the chess rules as well as the graphical representation of the chess game, will be provided by the `python-chess` library. Thus, this work concentrates on the actual implementation of artificial intelligence. 

Chess Engines that can outperform even the best human players, such as the latest `Stockfish` engine, usually use a combination of symbolic AI and machine learning to achieve the best results. Additionally, they also use high-performance computers. 
The objective is by no means here to compete with these engines, there will be no machine learning involved nor will the developed chess engine use a high-performance PC. Instead, the focus is on exploring various symbolic AI techniques and using chess as a demonstration for those. {cite}`stockfishGithub`

For the evaluation of chess positions, the (Simple Evaluation Function by Tomasz Michniewski)[https://www.chessprogramming.org/Simplified_Evaluation_Function] will be used. In combination with the alpha-beta-pruning algorithm and transposition tables, it forms the foundation of the chess engine. To achieve acceptable performance, the engine will be enhanced by using a quiescence search. 
Additional Enhancements such as the inclusion of chess books for openings and end games will be examined. 

## Structure

How this work is structured -> fill out later

# Project Setup

One great advantage of python is the number of already built-in features such as native JSON or YAML support. Nevertheless, in almost all python projects there is some need to install additional third-party libraries. Most of these are available in the python package index (PyPi) and can easily be installed with the tool `pip`. 
These dependencies might require a specific platform, python version or even other dependencies in some specific versions. Newer versions of the libraries might not always be compatible with older versions. In a collaborative project, it is therefore essential to specify the dependencies and their version. Pip has a built-in way to do this by writing all installed dependencies and their version to a requirements.txt file with the command `pip freeze > requirements.txt`. Another developer can then install exactly these dependencies with the command `pip install -r requirements.txt`. 

The main problem with this approach is that all dependencies will be installed globally and therefore might conflict with other packages. Additionally, `requirements.txt` will also contain all globally installed dependencies. To avoid these problems it is a common approach to create a virtual environment for each project. Some popular tools for creating and managing virtual environments are `virtualenv` and `pyenv`, both work well with `pip` and `requirements.txt`

Although this setup works in general it has the problem of being rather complex and error-prone. There are multiple tools needed and possibly documentation on how to use these in the project. Additionally, the workflow is not enforced by the tools, there is no need to name the file `requirements.txt` nor to even keep it up to date and therefore error-prone. Additional requirements such as building or publishing the package, although not relevant for this project, need additional tools. 

To deal with these issues there exist several tools operating on a higher level and combining the single tools. Some of the more popular tools are `Pipenv` and `Poetry`. Both are suitable for this project, but `Poetry` was chosen as it's already known by the authors. With poetry metadata and dependencies are listed in a `poetry.toml` file. The command `poetry install` will read the file and write a resolution of the dependencies and their version constraints to a `poetry.lock` file. Then it will create a new virtual environment or use a previously created one and install the resolved dependencies. If the `poetry.lock` file already exists the already resolved dependencies will be installed directly. To run a tool inside the virtual environment, use the `poetry run` command. For more detailed information on poetry consult the (documentation)[https://python-poetry.org/docs/]. 

By requirement, the chess engine will be built with jupyter notebook. To execute code inside a jupyter notebook a jupyter python kernel is needed. The kernel is a dependency of the project and will be installed in the virtual environment by poetry. Additionally, it might be necessary to generate a kernel description with `poetry run python -m ipykernel install --user --name poetryenv --display-name "Python (poetry env)"` so the kernel can be selected in jupyter.

# Introduction to python-chess

To concentrate on the actual implementation of the chess engine, the game itself and its representation are handled by the python-chess library. The most recent version 1.7.0, at the time of writing, is already specified as a dependency and will be installed in the virtual env. 

One of the core classes is `Board`, which specifies a single chess position and a move stack. The default constructor will create a board with the standard chess starting position and is graphically represented as ASCII when printed. 

In [ ]:
import chess

board = chess.Board()
print(board)

The board's interface allows working with the move stack by using standard stack operations such as `push`, `pop` or `peek`. The next example shows how to make moves with different chess notations. For the uci notation, which specifies the position of the piece and the target position, the `chess.Move.from_uci(uci: str)` can be used in combination with `push`. For the san notation, which, in the short form, specifies the piece and target position, the function `push_san` can be used. 

In [ ]:
board.push_san("e4")
board.push_san("e5")
board.push_san("Qh5")
board.push_san("Nc6")
board.push(chess.Move.from_uci("f1c4"))
board.push(chess.Move.from_uci("g8f6"))
board.push(chess.Move.from_uci("h5f7"))

Again, the current position can be printed as ASCII or instead as an svg image using the builtin display function.

In [ ]:
board

The above position is checkmate and is famously known as the Scholar’s mate. The library implements all chess rules and therefore allows to check that the above position is indeed checkmate with `board.is_checkmate()`. 
The board class offers a rich interface of functions and properties, such as `board.legal_moves` to get all allowed next moved or `board.is_stalemate()` to check for a stalemate, which will be introduced when needed. 
Additional features such as the inclusion of an opening book will be introduced as well when needed. 

# Random Chess Engine

The simplest possible chess engine is a random engine, 
which knows the chess rules and plays any available move. 
The goal is to incrementally enhance this basic chess engine 
and let different developed chess engines play against each other in the end. 


The `ScoredMove` data class represents a valid chess move along with a score.
By using the `dataclass` decorater with `order=True` python comparison methods, 
such as `__lt__()`, will be auto-generated.
This allows to compare two scored moves or sort a list of scored moves.
The comparison is only based on the score as the move field has `compare=False`. 

In [ ]:
from dataclasses import dataclass, field


@dataclass(order=True)
class ScoredMove:
    """Class for handling a move and its score."""
    score: int
    move: chess.Move = field(compare=False)

To simplify writing generic code for playing a chess game with different engines,
all engines will implement a common interface. 
The interface `Engine`,
which is highly inspired by `chess.engine`,
has two abstract functions: `play` and `analyse`.

The `play` function takes the current board as a parameter 
and returns a `chess.engine.PlayResult` Object, 
which contains the next move 
and information if the engine offered a draw or resigned. 

The `analyse` function takes the board as a parameter as well 
and returns a list of `ScoredMove` objects.

In [ ]:
from abc import ABC, abstractmethod
import chess.engine


class Engine(ABC):
    """Interface for chess engines"""
    @abstractmethod
    def play(self, board: chess.Board) -> chess.engine.PlayResult:
        pass

    @abstractmethod
    def analyse(self, board: chess.Board) -> list[ScoredMove]:
        pass

The `RandomEngine` will inherit from `Engine` 
and implement both abstract methods. 

The `play` method will randomly select one of the legal moves 
and return it as a `chess.engine.PlayResult`. 
The usage of `analyse` in the `play` method is not strictly necessary here,
but is the general pattern used by the following engines,
when `analyse` gets more complex.

The `RandomEngine` will inherit from `Engine` 
and implement both abstract functions. 
In the `play` function 
the engine will randomly select a move 
from the legal moves provided by the `board` object 
and return it as a `chess.engine.PlayResult`. 
The internal function `_getNextMoves` is used here 
to access the legal moves from the board. 
It's basically just a wrapper function around `board.legal_moves` 
and allows later engines to subclass and provide a different implementation.

The `analyse` function 
will return all possible legal moves 
with a score of zero for each as the random engine 
does not evaluate the moves.

In [ ]:
import random
import chess.engine


class RandomEngine(Engine):
    """Basic chess engine playing a random legal move"""
    def play(self, board: chess.Board) -> chess.engine.PlayResult:
        scoredMove = random.choice(self.analyse(board))
        return chess.engine.PlayResult(move=scoredMove.move, ponder=None)

    def analyse(self, board: chess.Board) -> list[ScoredMove]:
        return [ScoredMove(0, move) for move in board.legal_moves]

To let two engines 
inheriting from `Engine` 
play against each other 
the `playGame` function was created. 
It takes two engines as a parameter 
and let them move alternating 
until the game is over. 
If the game is over, 
this is detected by the board function `is_game_over`. 
The function then returns the board. 
Optionally
one can set the third parameter `displayBoard` to true 
so the board is printed 
after every move. 
Additionally, a move limit can be specified to stop after a specific number of moves.
Both are helpful for debugging.

In [ ]:
import IPython.display
import time
from typing import Optional

from datetime import datetime


def playGame(
    engine1: Engine,
    engine2: Engine,
    displayBoard: bool = False,
    logMoves: bool = False,
    moveLimit: Optional[int] = None
) -> chess.Board():
    """Function to play two Engines against each other
    
    Keyword arguments:
    displayBoard -- toggle to enable displaying the board after each half turn (default false)
    logMoves -- toggle to create a log of moves (default false)
    moveLimit -- optional limit of moves (default None)
    """
    board = chess.Board()
    engineTurn = engine1

    logFileName = f"../log/{datetime.isoformat(datetime.now())}.txt"

    if displayBoard:
        IPython.display.display(board)

    while not board.is_game_over():
        move = engineTurn.play(board).move

        if logMoves:
            with open(logFileName, "a") as f:
                if board.turn is chess.WHITE:
                    f.write(board.lan(move) + "\t")
                else:
                    f.write(board.lan(move) + "\n")

        board.push(move)

        if engineTurn is engine1:
            engineTurn = engine2
        else:
            engineTurn = engine1

        if displayBoard:
            IPython.display.clear_output(wait=True)
            IPython.display.display(board)
            time.sleep(0.5)

        if moveLimit and len(board.move_stack) >= moveLimit:
            break
    return board

Next, 
the function is called 
with two `RandomEngine` objects 
and then prints the outcome of the board 
and the end position. 
When passsing `displayBoard=True` 
one can see that both engines play without any strategy.

The random number generator (RNG) depends on an inital seed. 
In the code the seed is explicitly set to create reproducible result.
For real world applications, the seed should be set by a high entropy source.

In [ ]:
random.seed(42)
board = playGame(RandomEngine(), RandomEngine())
print(board.outcome())
board

# Enhanced Random Engine

## Add an opening book 

The first improved version of `RandomEngine` 
is the `OpeningRandomEngine`,
which will have access to an opening book. 
(Polyglot)[http://hgm.nubati.net/book_format.html] is a widespread book format for openings.
Because a lot of open-source opening books 
in the `polyglot` format 
can be found on the internet
and because it is supported by the `python-chess` library,
it is used in this example. 
The repository contains some sample `polyglot` opening books.

The `OpeningRandomEngine` 
inherits from `RandomEngine` 
and overwrites the `play` method. 
In the new `play` method, 
the engine first searches for moves 
in the opening book 
using the helper method `getOpeningMoves` 
and,
if no move is found,
falls back to the implementation of `RandomEngine`. 
The method `getOpeningMoves` 
takes the current chessboard as a parameter 
and returns a list of `ScoredMove` objects. 
It uses the `chess.polyglot.MemoryMappedReader` class 
of the `python-chess` library
to open a `polyglot` opening book. 
The `find_all` method of the reader 
is used to search the opening book 
and create a list of scored moves. 

Notice, 
however, 
that the score of the move is defined by the author of the opening book and usually depends on how often and sucessfull it was played by profiecient players. 
In some rare cases, following the book, leads quickly into a draw by fivefold repition, therefore a minimum weight has been defined.

In [ ]:
import random
import chess.polyglot


class OpeningRandomEngine(RandomEngine):
    """Chess engine using opening books to make better moves"""
    MIN_WEIGHT = 50

    def getOpeningMoves(self, board: chess.Board) -> list[ScoredMove]:
        with chess.polyglot.open_reader("../data/polyglot/ProDeo.bin") as reader:
            nextMoves = [
                ScoredMove(entry.weight, entry.move)
                for entry in reader.find_all(board)
                if entry.weight >= self.MIN_WEIGHT
            ]

        return nextMoves

    def play(self, board: chess.Board) -> chess.engine.PlayResult:
        scoredMoves = self.getOpeningMoves(board) or self.analyse(board)
        scoredMove = random.choice(scoredMoves)
        return chess.engine.PlayResult(move=scoredMove.move, ponder=None)

The `OpeningRandomEngine` can now play against the `RandomEngine`. 

In [ ]:
random.seed(42)
board = playGame(OpeningRandomEngine(), RandomEngine())
print(board.outcome())
board

The `OpeningRandomEngine` 
is not expected to win 
against the `RandomEngine` in general. 
The reason for this is 
the `RandomEngine` often playing bad moves 
such as `Sh6` after `d4`, 
which are usually not included in an opening book. 
The below code shows 
how to calculate the probability 
of randomly hitting the first opening move.

In [ ]:
board = chess.Board()
engine = OpeningRandomEngine()
openingMoves = engine.getOpeningMoves(board)
probabilityFirstMove = len(openingMoves) / len(list(board.legal_moves))
print(f"Chance to hit opening move after the first move: {probabilityFirstMove * 100}%")

To calculate the probability 
of hitting two moves in a row of the opening book 
by chance
all possible situations after the first opening move have to be examined. 
The script 
takes the average 
of possible opening moves and legal moves 
and uses those to calculate the probability.

In [ ]:
import statistics

legalMovesSecondMove = []
openingMovesSecondMove = []
for scoredMove in openingMoves:
    board.push(scoredMove.move)
    legalMovesSecondMove.append(len(list(board.legal_moves)))
    openingMovesSecondMove.append(len(engine.getOpeningMoves(board)))
    board.pop()

probabilitySecondMove = probabilityFirstMove * (
    statistics.mean(openingMovesSecondMove) / statistics.mean(legalMovesSecondMove)
)

print(f"Chance to hit opening move two times in a row: {probabilitySecondMove * 100}%")

To make sure 
the `OpeningRandomEngine` does only play moves 
from the opening books, 
multiple tests check the engine's choices
against the available opening moves for different seeds.

In [ ]:
engine = OpeningRandomEngine()

# Moves in the ProDeo Opening Book
firstMoves = [
    chess.Move.from_uci("d2d4"), chess.Move.from_uci("e2e4"), chess.Move.from_uci("c2c4"), chess.Move.from_uci("g1f3")
]

random.seed(0)
board = chess.Board()
playResult = engine.play(board)
assert playResult.move in firstMoves, f"Random opening engine chose the move {playResult.move} not in the opening book"

random.seed(42)
board = chess.Board()
playResult = engine.play(board)
assert playResult.move in firstMoves, f"Random opening engine chose the move {playResult.move} not in the opening book"

# Add first evaluation function

The next engine 
is the first one
to evaluate chess positions 
to find better moves.

Instead of creating the `AbsoluteEnhancedRandomEngine` as one block of code,
a slightly different approach will be taken here:
The class is created 
and functions will be replaced or added later.
This enables a dedicated explanation of each method seperately
before writing the code,
instead of having to explain everything beforehand
and loosing the context.
This form of dynamic class modification is called monkey patching in the python community and is often used as a technique to patch third party code {cite}`plone:glossary`.

In [ ]:
class AbsoluteEnhancedRandomEngine(OpeningRandomEngine):
    """Chess engine using the absolute value of the chessboard to evalute moves"""


A simple way of evaluating a move 
is based on absolute values for each piece.
The function `pieceValue` 
takes a chess piece as input 
and returns an integer 
representing its value as commonly defined in {cite}`Capablanca2006` and other books.
For better readability 
a set of constants 
is introduced for these values.

In [ ]:
AbsoluteEnhancedRandomEngine.VALUE_PAWN = 1
AbsoluteEnhancedRandomEngine.VALUE_KNIGHT = 3
AbsoluteEnhancedRandomEngine.VALUE_BISHOP = 3
AbsoluteEnhancedRandomEngine.VALUE_ROOK = 5
AbsoluteEnhancedRandomEngine.VALUE_QUEEN = 9
AbsoluteEnhancedRandomEngine.VALUE_KING = 0


def pieceValue(self: Engine, pieceType: chess.PieceType) -> int:
    if pieceType is chess.PAWN:
        return self.VALUE_PAWN
    if pieceType is chess.KNIGHT:
        return self.VALUE_KNIGHT
    if pieceType is chess.BISHOP:
        return self.VALUE_BISHOP
    if pieceType is chess.ROOK:
        return self.VALUE_ROOK
    if pieceType is chess.QUEEN:
        return self.VALUE_QUEEN
    if pieceType is chess.KING:
        return self.VALUE_KING  # King is always needed and has no material value
    assert False, f"There is no piece value for the piece {pieceType}"


AbsoluteEnhancedRandomEngine.pieceValue = pieceValue

The evaluation
of the game 
is determined by summing up the piece values of all the white pieces
and subtracting the piece values of all the black pieces. 
A positive score therefore is good for White whereas a negative score is good for Black. 
This is implemented in the method `_absolute_heuristic`,
which takes a board as input 
and returns an integer representing this evaluation. 

In [ ]:
def _absolute_heuristic(self, board: chess.Board) -> int:
    """Calculate the value of all pieces on the board"""

    score = 0
    for square in chess.SquareSet(board.occupied_co[chess.WHITE]):
        score += self.pieceValue(board.piece_type_at(square))

    for square in chess.SquareSet(board.occupied_co[chess.BLACK]):
        score -= self.pieceValue(board.piece_type_at(square))

    return score


AbsoluteEnhancedRandomEngine._absolute_heuristic = _absolute_heuristic

To have an incentive to win,
the value of a winning board 
is valued with 100.
As a draw has no advantage for either side
it is valued with 0.
For better readability 
these values are also stored in constants.

In [ ]:
AbsoluteEnhancedRandomEngine.VALUE_CHECKMATE = 100
AbsoluteEnhancedRandomEngine.VALUE_DRAW = 0

There exist multiple draw conditions, 
so for ease of use, 
they are combined into one function, 
which is added to the `chess.Board` class afterwards.

In [ ]:
def is_draw(self) -> bool:
    """Function to check for all automatically enforced draw conditions"""
    return self.is_stalemate() or self.is_insufficient_material() or self.is_fivefold_repetition(
    ) or self.is_seventyfive_moves()


chess.Board.is_draw = is_draw

Next, the `analyse` method can be overriden to use the newly created evaluation function. 
All legal moves - or rather the resulting next boards - are evaluated by first checking
for checkmate or draw, or otherwise by using the previously implemented `_absolute_heuristic` method.

To make the code easier to understand
this is logic is implemented in the `_evaluate_move` method.

The possible moves are then shuffled and, afterwards, sorted by their score, 
creating a different order of moves having the same score, 
depending on the state of the RNG.

By default, the python `sort` method sorts from lowest to highest. 
Therefore, the first move is the best for Black, unless the sorting order is reversed.

In [ ]:
def _evaluate_move(self: Engine, board: chess.Board, move: chess.Move) -> int:
    """Evaluate a single move using _absolute_heuristic"""
    whitesTurn = board.turn is chess.WHITE
    board.push(move)
    if board.is_checkmate():
        score = self.VALUE_CHECKMATE if whitesTurn else -self.VALUE_CHECKMATE
    elif board.is_draw():
        score = self.VALUE_DRAW
    else:
        score = self._absolute_heuristic(board)
    board.pop()
    return score


AbsoluteEnhancedRandomEngine._evaluate_move = _evaluate_move


def analyse(self, board: chess.Board) -> list[ScoredMove]:
    """Analyse method using _absolute_heuristic"""
    whitesTurn = board.turn is chess.WHITE
    nextMoves = []
    for move in board.legal_moves:
        score = self._evaluate_move(board, move)
        nextMoves.append(ScoredMove(score, move))

    random.shuffle(nextMoves)
    nextMoves.sort(reverse=whitesTurn)
    return nextMoves


AbsoluteEnhancedRandomEngine.analyse = analyse

To use the advantage of having a score
the `play` method is modified to choose a random move from the top two moves 
returned by the `analyse` method.

In [ ]:
def play(self, board: chess.Board) -> chess.engine.PlayResult:
    scoredMoves = self.getOpeningMoves(board) or self.analyse(board)
    scoredMove = random.choice(scoredMoves[:2])
    return chess.engine.PlayResult(move=scoredMove.move, ponder=None)


AbsoluteEnhancedRandomEngine.play = play

The `AbsoluteEnhancedRandomEngine` strategy is to capture everything the engine can. 
Additionally, it is also able to find mate in one. 
This allows the engine to win most of the time against the `OpeningRandomEngine`, although it is not guaranteed.

TO DO: Add a statistic?

In [ ]:
random.seed(42)
board_absolute = playGame(OpeningRandomEngine(), AbsoluteEnhancedRandomEngine(), displayBoard=False)
IPython.display.clear_output(wait=True)
IPython.display.display(board_absolute)
print(board_absolute.outcome())

To verfify the correctness the `_absolute_heuristic` its output is checked against some famous chess games.

In [ ]:
random.seed(42)
engine = AbsoluteEnhancedRandomEngine()

# Topalov, Veselin (2740) vs. Shirov, Alexei (2710)
SHIROV_SACRIFICE = "8/8/4kpp1/3p1b2/p6P/2B5/6P1/6K1 b - - 0 47"
board = chess.Board(SHIROV_SACRIFICE)
# IPython.display.display(board)
assert engine._absolute_heuristic(board) == -2, "Evaluation of position is not -2"

# Evgeny Yuryevich Vladimirov vs. Vladimir Viktorovich Epishin
VLADIMIROV_THUNDERBOLT = "r4k1r/1b2bPR1/p4n2/3p4/4P2P/1q2B2B/PpP5/1K4R1 w - - 0 26"
board = chess.Board(VLADIMIROV_THUNDERBOLT)
# IPython.display.display(board)
assert engine._absolute_heuristic(board) == -10, "Evaluation of position is not -10"

Additionally, a special chess position was constructed to test the `analyse` method. In this position, different moves such as capturings and promotions are possible.

In [ ]:
random.seed(42)
PROMOTION_POSITION = "Kn2rn1k/1p2P3/8/8/8/8/8/8 w - - 0 1"
board = chess.Board(PROMOTION_POSITION)
# IPython.display.display(board)
assert engine._absolute_heuristic(board) == -11, "Evaluation of position is not -11"
result = engine.analyse(board)
assert result == [ScoredMove(0, chess.Move.from_uci("e7f8q")),
                  ScoredMove(-4, chess.Move.from_uci("e7f8r")),
                  ScoredMove(-6, chess.Move.from_uci("e7f8b")),
                  ScoredMove(-6, chess.Move.from_uci("e7f8n")),
                  ScoredMove(-10, chess.Move.from_uci("a8b7")),
                  ScoredMove(-11, chess.Move.from_uci("a8b7"))], f'{result} is not correct'

The `AbsoluteEnhancedRandomEngine.analyse` method
is not very effective as it always recalculates the whole board. Thus, it does up to 16 calculations. 
With one move, the evaluation can change in the following ways: 
* There is a checkmate
* There is a draw
* Capture of a piece
* Promotion of a piece
* Capture and promotion of a piece
* Nothing changes

With this idea in mind, the next engine implements an incremental heuristic.

In [ ]:
class RelativeEnhancedRandomEngine(AbsoluteEnhancedRandomEngine):
    """Chess engine using the change of value of the chessboard to evalute moves"""

The new method `_relative_heuristic` takes a board and a move as a parameter and returns the change of the score by playing that move from the perspective of the player to move.  
If a figure is captured, 
the overall score of the board 
will increase 
by its value.
If a pawn is promoted, 
the overall score of the board 
will be increased 
by the value of the promoted piece 
minus the value of the pawn.
The score is returned as is if it is White's turn, otherwise it is returned negated.

In [ ]:
def _relative_heuristic(self, board: chess.Board, move: chess.Move) -> int:
    value = 0
    if capturedPiece := board.piece_type_at(move.to_square):
        value += self.pieceValue(capturedPiece)
    if promotionPiece := move.promotion:
        value += self.pieceValue(promotionPiece) - self.VALUE_PAWN

    return value if board.turn is chess.WHITE else -value


RelativeEnhancedRandomEngine._relative_heuristic = _relative_heuristic

The `analyse` method is similar to the one defined for the `AbsoluteEnhancedRandomEngine` class. 
The only difference is 
that the initial score of the board 
is calculated first by using the `_absolute_heuristic` method
and the `_evaluate_move` method taking an additional parameter `currentScore`. 
In the loop, the value of each move is the sum of the inital score and the value calculated by `_relative_heuristic` of the move.

In [ ]:
def _evaluate_move(self: Engine, board: chess.Board, move: chess.Move, currentScore: int) -> int:
    """Evaluate a single move using _relative_heuristic"""
    whitesTurn = board.turn is chess.WHITE
    score = currentScore + self._relative_heuristic(board, move)
    board.push(move)
    if board.is_checkmate():
        score = self.VALUE_CHECKMATE if whitesTurn else -self.VALUE_CHECKMATE
    elif board.is_draw():
        score = self.VALUE_DRAW
    board.pop()
    return score


RelativeEnhancedRandomEngine._evaluate_move = _evaluate_move


def analyse(self, board: chess.Board) -> list[ScoredMove]:
    """Analyse method using _relative_heuristic"""
    currentScore = self._absolute_heuristic(board)
    whitesTurn = board.turn is chess.WHITE
    nextMoves = []
    for move in board.legal_moves:
        score = self._evaluate_move(board, move, currentScore)

        nextMoves.append(ScoredMove(score, move))

    random.shuffle(nextMoves)
    nextMoves.sort(reverse=whitesTurn)
    return nextMoves


RelativeEnhancedRandomEngine.analyse = analyse

As both engines should play the same move, the outcome of the game against the `OpeningRandomEngine` is the same provided the seed is the same.

In [ ]:
random.seed(42)
board_relative = playGame(OpeningRandomEngine(), RelativeEnhancedRandomEngine(), displayBoard=False)
IPython.display.clear_output(wait=True)
IPython.display.display(board)
print(board_relative.outcome())

To verfify that both engines play exactly the same moves, the following test compares the move stacks of boths games.

In [ ]:
assert board_relative.move_stack == board_absolute.move_stack, f"There were different moves"

Performance measurements

TO DO: 
- More details (statistics)

In [ ]:
random.seed(42)
absoluteEngine = AbsoluteEnhancedRandomEngine()
relativeEngine = RelativeEnhancedRandomEngine()

board1 = chess.Board(SHIROV_SACRIFICE)
board2 = chess.Board(VLADIMIROV_THUNDERBOLT)
board3 = chess.Board(PROMOTION_POSITION)

print("Absolute Engine")
random.seed(42)
%timeit -r 2 -n 5 absoluteEngine.analyse(board1)
%timeit -r 2 -n 5 absoluteEngine.analyse(board2)
%timeit -r 2 -n 5 absoluteEngine.analyse(board3)

print("Relative Engine")
random.seed(42)
%timeit -r 2 -n 5 relativeEngine.analyse(board1)
%timeit -r 2 -n 5 relativeEngine.analyse(board2)
%timeit -r 2 -n 5 relativeEngine.analyse(board3)

## A human engine

To enable human players to play against the chess engines 
a `HumanEngine` is created, 
which is compatible with the previously defined `playGame` method.

An array of keywords to abort the game 
is defined,
so a player is not forced 
to manually interrupt the kernel
to do so.

Instead of implementing a dedicated `analyse` function 
as it is required by the `Engine` interface
it is "copied" from the `AbsoluteEnhancedRandomEngine`.

In [ ]:
class HumanEngine(Engine):
    """Chess engine using the abilities of a human player"""
    """ List of keywords to abort input"""
    ABORT_KEYWORDS = [
        "EXIT",
        "CANCEL",
        "",
    ]

    def play(self, board: chess.Board) -> chess.engine.PlayResult:
        """Play method taking input of human players"""
        whitesTurn = board.turn is chess.WHITE
        legalMoves = board.legal_moves
        while True:
            enteredMove = input(f"Enter move for {'white' if whitesTurn else 'black'} in UCI notation: ")
            if enteredMove.upper() in HumanEngine.ABORT_KEYWORDS:
                raise Exception("User aborted")
            try:
                move = chess.Move.from_uci(enteredMove)
                if move in legalMoves:
                    return ScoredMove(0, move)
                print("Illegal input.  Please try again.")
            except ValueError:
                print(f"Illegal notation. Valid moves are {list(legalMoves)}")

    def analyse(self, board: chess.Board) -> list[ScoredMove]:
        pass

This engine can be used as shown in the following commented block.

In [ ]:
# board_human = playGame(HumanEngine(), RelativeEnhancedRandomEngine(), displayBoard=True)
# print(board.outcome())

## Some Stockfish Experiments!

In [ ]:
# import chess
# import chess.engine

# engine = chess.engine.SimpleEngine.popen_uci(r"../data/stockfish_14.1_linux_x64/stockfish_14.1_linux_x64")

# board = chess.Board()
# while not board.is_game_over():
#     result = engine.play(board, chess.engine.Limit(time=0.1))
#     board.push(result.move)

# engine.quit()

# board

To use an existing UCI compatible engine, it can be loaded as follows:

In [ ]:
engine = chess.engine.SimpleEngine.popen_uci(r"../data/stockfish_14.1_linux_x64/stockfish_14.1_linux_x64")

In [ ]:
engine.analyse(chess.Board(SHIROV_SACRIFICE), chess.engine.Limit(time=0.1))

In [ ]:
engine.analyse(chess.Board(VLADIMIROV_THUNDERBOLT), chess.engine.Limit(time=0.1))